In [ ]:
import cv2
import tensorflow as tf
import os
import numpy as np
import network as net
from Training import openpose
import matplotlib.pyplot as plt
from scipy.ndimage.filters import maximum_filter
import math
from collections import Counter, defaultdict
import itertools
import matplotlib as mpl
import time
import ast

In [ ]:
path, joint = load_data()
def make_heatmap(batch_anno_data, width = 44, height = 44, num_of_maps = 17):
        batch_size = 1
        width = 44
        height = 44
        num_of_maps = 17
        output = np.zeros((batch_size, width, height, num_of_maps))
        for index, joint_data in enumerate(batch_anno_data):

            heatmap = np.zeros((width, height, num_of_maps), np.int16)#batch 일단 뺌

            for joints in joint_data:
                buffer = list(joints.items())
                key_buffer = joints.keys()

                for i in range(len(buffer)):
                    buffer[i] = list(buffer[i])
                    buffer[i][0] = int(buffer[i][0])
                bubble_sort(buffer)

                idx = 0
                for j in range(17):

                    #print(j)
                    if('%d' %j in key_buffer):
                        center_x = buffer[idx][1][0]
                        center_y = buffer[idx][1][1]
                        #joint = buffer[idx][1]
                        joint = [center_y, center_x]
                        idx = idx + 1
                        _put_heatmap_on_plane(heatmap, plane_idx = j, joint = joint, sigma = 3, height = height, \
                                              width = width, stride = 1)
                    else:
                        pass
                idx = 0
            heatmap[:, :, -1] = np.clip(1.0 - np.amax(heatmap, axis=2), 0.0, 1.0)
            output[index] = heatmap
        return output
    
    
def _put_heatmap_on_plane(heatmap, plane_idx, joint, sigma, height, width, stride):
    start = stride / 2.0 - 0.5

    center_x, center_y = joint

    for g_y in range(height):
        for g_x in range(width):
            x = start + g_x * stride
            y = start + g_y * stride
            d2 = (x-center_x) * (x-center_x) + (y-center_y) * (y-center_y)
            exponent = d2 / 2.0 / sigma / sigma
            if exponent > 4.6052:
                continue

            heatmap[g_y, g_x, plane_idx] += math.exp(-exponent)
            if heatmap[g_y, g_x, plane_idx] > 1.0:
                heatmap[g_y, g_x, plane_idx] = 1.0
def bubble_sort(L):
    for i in range(len(L)-1):
        for j in range(len(L)-1):
            if L[j] > L[j+1]:
                temp = L[j+1]
                L[j+1] = L[j]
                L[j] = temp
                
                
def _put_paf_on_plane(vectormap, countmap, plane_idx, center_from, center_to, threshold, height, width, stride):
    center_from = (center_from[0] // stride, center_from[1] // stride)
    center_to = (center_to[0] // stride, center_to[1] // stride)

    vec_x = center_to[0] - center_from[0]
    vec_y = center_to[1] - center_from[1]

    min_x = max(0, int(min(center_from[0], center_to[0]) - threshold))
    min_y = max(0, int(min(center_from[1], center_to[1]) - threshold))

    max_x = min(width, int(max(center_from[0], center_to[0]) + threshold))
    max_y = min(height, int(max(center_from[1], center_to[1]) + threshold))

    norm = math.sqrt(vec_x ** 2 + vec_y ** 2)
    if norm < 1e-8: #1e-8 이하는 0으로 인식되서 0으로 나눌수 없다는 에러 발생. 따라서 return처리 해줌
        return

    vec_x /= norm
    vec_y /= norm
    
    for x in range(min_x, max_x):
        for y in range(min_y, max_y):
            bec_x = x - center_from[0]
            bec_y = y - center_from[1]
            dist = abs(bec_x * vec_y - bec_y * vec_x)

            if dist > threshold:
                continue

            countmap[x][y][plane_idx] = countmap[x][y][plane_idx] + 1

            vectormap[x][y][plane_idx*2+0] = vec_x
            vectormap[x][y][plane_idx*2+1] = vec_y
            
def make_paf_field(batch_joint_data, width = 44, height = 44, num_of_maps = 17):
        batch_size = 1
        output1 = np.zeros((batch_size, width, height, num_of_maps*2))
        output2 = np.zeros((batch_size, width, height, num_of_maps))
        for index, joint_data in enumerate(batch_joint_data):
            joint_pairs = list(zip(
                [9, 8, 8, 8,13,14,12,11,7,6,6,3,4,2,1],
                [8,13,12, 7,14,15,11,10,6,3,2,4,5,1,0]))
            #make vector map
            width = 44
            height = 44
            num_of_maps = 17
            vectormap = np.zeros((width, height, num_of_maps*2), dtype=np.float32)#batch 일단 뺌
            countmap = np.zeros((width, height, num_of_maps), np.int16)#batch 일단 뺌

            for joints in joint_data:
                key = (joints.keys())
                for plane_idx, (j_idx1, j_idx2) in enumerate(joint_pairs):
                    if(('%d' %j_idx1 in key) and ('%d' %j_idx2 in key)):

                        center_from = joints['%d'%j_idx1]
                        center_to = joints['%d'%j_idx2]

                        if not center_from or not center_to:
                            continue
                        _put_paf_on_plane(vectormap=vectormap, countmap=countmap, plane_idx=plane_idx, center_from=center_from, center_to=center_to, \
                                          threshold=1, height=44, width=44, stride = 1)

            nonzeros = np.nonzero(countmap)


            for x, y, p in zip(nonzeros[0], nonzeros[1], nonzeros[2]):
                if countmap[x][y][p] <= 0:
                    continue
                vectormap[x][y][p*2+0] /= countmap[x][y][p]
                vectormap[x][y][p*2+1] /= countmap[x][y][p]

            output1[index] = vectormap.astype(np.float32)
            output2[index] = countmap
        return output1, output2 #output1 -> vectormap, output2 -> countmap
    
heatmap___ = make_heatmap([joint[0]], 44, 44, 17)

print(np.shape(heatmap___))
heatmap___transpose = np.transpose(heatmap___[0], (2, 0, 1))
heatmap___transpose_sum = np.zeros((44, 44))
for i in range(17):
    plt.imshow(heatmap___transpose[i])
    plt.show()
for i in range(16):
    heatmap___transpose_sum = heatmap___transpose_sum + heatmap___transpose[i]
plt.imshow(heatmap___transpose_sum)
plt.show()

vectormap___, countmap = make_paf_field([joint[0]], 44, 44, 17)

print(np.shape(vectormap___[0]))
vectormap___transpose = np.transpose(vectormap___[0], [2, 0, 1])
vectormap___transpose_sum = np.zeros((44, 44))
for i in range(34):
    plt.imshow(vectormap___transpose[i])
    plt.show()
for i in range(34):
    vectormap___transpose_sum = vectormap___transpose_sum + vectormap___transpose[i]
print("summed vectormap")
plt.imshow(vectormap___transpose_sum)
plt.show()

NMS_Threshold = 0.1
heatmap___transpose = heatmap___transpose - heatmap___transpose.min(axis=1).min(axis=1).reshape(17, 1, 1)
heatmap___transpose = heatmap___transpose - heatmap___transpose.min(axis=2).reshape(17, heatmap.shape[1], 1)
_NMS_Threshold = max(np.average(heatmap___transpose) * 4.0, NMS_Threshold)
_NMS_Threshold = min(_NMS_Threshold, 0.3)

nms_heatmap = np.zeros((17, 44, 44))
nms_heatmap_sum = np.zeros((44, 44))
coords = []
for i in range(17-1):#except background
    nms_heatmap[i] = non_max_suppression(heatmap___transpose[i], window_size=7, threshold=_NMS_Threshold)
    coords.append(np.where(nms_heatmap[i] >= _NMS_Threshold))
    plt.imshow(nms_heatmap[i])
    plt.show()
    
for i in range(16):
    nms_heatmap_sum = nms_heatmap_sum + nms_heatmap[i]
    
plt.imshow(nms_heatmap_sum)
plt.show()

batch_size = 1
test_img_path = load_test_data("./MPII_Dataset/test_image_demo/")

batch_img_path = make_test_batch(img_path = test_img_path, batch_size = batch_size)
batch_img = path_to_image(batch_img_path, batch_size)

batch_img = batch_img.reshape([356, 356, 3])
buffer = batch_img[:,:,[2,1,0]]
preprocessed = preprocess(buffer, 356, 356)
preprocessed = preprocessed[:, :, [2,1,0]]
plt.imshow(buffer)
plt.show()

humans = estimate_pose(coords, nms_heatmap, vectormap___transpose)
cb_showimg(buffer, preprocessed, nms_heatmap, vectormap___transpose, humans, show_process=True)